#### Data Merging

This file is to merge data from purple air sensor. <br>
Purple air sensor makes separate csv file for each day. This code performs merge on those.

In [ ]:
import pandas as pd
import glob
import os
import re

import re
import pandas as pd
import os
import pytz
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import warnings
from concurrent.futures import ThreadPoolExecutor

#### Data Merging

In [ ]:

def merge(dir):
    """Function to merge data
    args:
        dir: folder directory containing csv files
    
    """

    print(f'Merging for {dir}')
    print('='*50)

    path = dir

    # Print the path for debugging
    print(f"Looking for CSV files in: {path}")

    # Use glob to get all the CSV files in the directory
    all_files = glob.glob(os.path.join(path, "*.csv"))


    # Initialize an empty list to hold the DataFrames
    df_list = []

    # Loop through the list of files and read each file into a DataFrame
    print("Reading file...")
    for file in tqdm(all_files):
        try:
            df = pd.read_csv(file, encoding='ascii')
            df_list.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")

    # Check if df_list is still empty after the loop
    if len(df_list) == 0:
        raise ValueError("No DataFrames were created. Check if the CSV files are valid and readable.")

    # Concatenate all the DataFrames in the list into a single DataFrame
    try:
        merged_df = pd.concat(df_list, ignore_index=True)
    except Exception as e:
        raise ValueError(f"Error concatenating DataFrames: {e}")

    merged_df['UTCDateTime'] = pd.to_datetime(merged_df['UTCDateTime'])
    
    target_dir = "merged"
    file_path = os.path.join(target_dir, f"{dir.split('\\')[-1]}.csv")
    try:
        merged_df.to_csv(file_path, index = False)
    except Exception as e:
        print("Error in saving file...")
        print(e)
        while True:
            print("1. Retry")
            print("2. Skip")
            option = int(input())
            if option == 1:
                try: 
                    merged_df.to_csv(file_path, index = False)
                except Exception as e:
                    print("Error again", e)
                    continue
            elif option == 2:
                break
            else:
                print("invalid input")
                continue


    print(f'file saved : {file_path}')

    # length = merged_df.shape[0]
    # atmdata = length - merged_df['pm2_5_atm_avg'].isna().sum()
    # cfdata = length - merged_df['pm2_5_cf_1_avg'].isna().sum()

    # return atmdata/length*100, cfdata/length*100, length


# merge("Ajimpur home data", "indoor Sensor No 12")

In [ ]:
dir = 'data'
locations = [f for f in os.listdir(dir) if os.path.isdir(os.path.join(dir, f))]



for location in tqdm(locations):
        if os.path.exists(os.path.join("merged", f"{location}.csv")):
            print(f"File already exists for {location}, skipping...")
            continue
        merge(os.path.join(dir, location))
